# SPC Coastal Fisheries Web Application - API Demo

This notebook demonstrates how to interact with the **SPC (Pacific Community) Coastal Fisheries Web Application** API to retrieve and analyze landing survey data.

1. **Authenticate** with the SPC Coastal Fisheries API
2. **Explore** available authorities, surveys, and species
3. **Query** landing survey reports
4. **Export** data in JSON and CSV formats
5. **Filter** data by specific criteria

## Prerequisites

Before running this notebook, ensure you have:
- Created a `.env` file with your SPC portal credentials
- Created a `.venv` and activate it (check how to do that in `READM.md` file)
- Installed required packages (`pip install -r requirements.txt`) in the python version runnin in your `.venv`
- Valid access to the SPC Coastal Fisheries Web Application

> **Note:** If you don't have credentials yet, contact Franck Magron, Olivier Loyau, or the Coastal Fisheries team to request access to the [SPC Coastal Fisheries Portal](https://www.spc.int/CoastalFisheries/).

## Import Libraries and Setup Environment

First, we'll import the necessary Python libraries and load our environment variables containing the API credentials.

In [ ]:
# Import required libraries for API interaction and data manipulation
import os
from dotenv import load_dotenv  # For loading environment variables from .env file
import requests                 # For making HTTP API calls
import pandas as pd            # For data analysis and manipulation

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
user = os.getenv("USER")
password = os.getenv("PASSWORD")
baseUrl = os.getenv("BASEURL")

# Verify that all required environment variables are loaded
if not all([user, password, baseUrl]):
    print("❌ Error: Missing environment variables!")
    print("Please check your .env file contains USER, PASSWORD, and BASEURL")
else:
    print("✅ All environment variables loaded successfully")

## Step 1: Authentication

Now we'll authenticate with the SPC Coastal Fisheries API using the `/account/SignIn` endpoint. This establishes a session and provides cookies for subsequent API calls.

In [ ]:
# Authenticate with the SPC Coastal Fisheries API

connection = requests.post(baseUrl + 'account/SignIn', data={
    "username": user, 
    "password": password
})

# Check if authentication was successful
if connection.json():
    print("Authentication successful!")
else:
    print("❌ Authentication failed!")
    print("Please check your credentials in the .env file")
    print(f"Status Code: {connection.status_code}")
    exit()

# Store session for reuse in other API calls
session_cookies = connection.cookies
print("Session cookies stored for future API requests")

## Step 2: Explore Available Authorities

Authorities represent different regions or organizations that manage coastal fisheries. Let's see which authorities your account can access.

In [ ]:
authorities = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/GetAuthorities',
    data={}, 
    cookies=session_cookies
).json()

# Convert to DataFrame for easier analysis
authorities_df = pd.DataFrame(authorities)

print(f"Response: Retrieved {len(authorities)} authorities")
print("\n First few authorities:")
print(authorities_df.head())


In [ ]:
# Analyze the unique authorities available
unique_auths = authorities_df['Name'].unique()

print(f" Your account has access to {len(unique_auths)} unique authorities:")
print("=" * 60)

# Display authorities in a formatted way
for i, auth in enumerate(unique_auths, 1):
    print(f"{i:2d}. {auth}")

print("=" * 60)

# Store the first authority for use in subsequent examples
first_authority = unique_auths[0] if len(unique_auths) > 0 else None
if first_authority:
    print(f"We'll use '{first_authority}' as our example authority for the following demonstrations")
else:
    print("⚠️ No authorities found for this account")

## Step 3: Discover Available Surveys

Explore what surveys are available for the example authority (above).

In [ ]:
# Get surveys for the first authority
print(f"Fetching surveys for authority: '{first_authority}'")
print("API Call: GET /FieldSurveys/LdsStatistics/GetSurveys")

surveys = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/GetSurveys',
    data={"authorities": [first_authority]},  # Filter by specific authority
    cookies=session_cookies
).json()

# Convert to DataFrame for analysis
surveys_df = pd.DataFrame(surveys)
unique_surveys = surveys_df['Name'].unique()

print(f"📈 Found {len(unique_surveys)} surveys for '{first_authority}':")
print("=" * 60)

# Display surveys in a formatted way
for i, survey in enumerate(unique_surveys, 1):
    print(f"{i:2d}. {survey}")


## Step 4: Explore Available Species

Let's discover what fish species are recorded in the surveys for our example authority.

In [ ]:
# Get species information for the first authority
print(f"Fetching species data for authority: '{first_authority}'")

species = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/GetSpecies',
    data={"authorities": [first_authority]},  # Filter by specific authority
    cookies=session_cookies
).json()

# Convert to DataFrame for analysis
species_df = pd.DataFrame(species)

# Analyze species groups and individual species
unique_species_groups = species_df['SpeciesGroup'].unique()
unique_species = species_df['Species'].unique()

print(f" Found {len(unique_species)} unique species organized into {len(unique_species_groups)} species groups")
print("\n Species Groups:")
print("=" * 50)

for i, group in enumerate(unique_species_groups, 1):
    species_in_group = species_df[species_df['SpeciesGroup'] == group]['Species'].nunique()
    print(f"{i:2d}. {group} ({species_in_group} species)")

print("=" * 50)



## Step 5: Discover Available Reports

Reports are predefined data structures that organize the survey data in different ways. Let's see what reports are available.

In [ ]:
# Get list of available reports
print("Fetching available landing survey reports...")
print("API Call: GET /FieldSurveys/LdsStatistics/GetReports")

reports = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/GetReports',
    data={}, 
    cookies=session_cookies
).json()

# Convert to DataFrame for analysis
reports_df = pd.DataFrame(reports)
unique_reports = reports_df['Name_EN'].unique()

print(f" Found {len(reports_df)} available reports:")
print("=" * 80)

# Display reports with their IDs for reference
for i, (idx, report) in enumerate(reports_df.iterrows(), 1):
    print(f"{i:2d}. {report['Name_EN']}")

print("=" * 80)

# Store the first report for demonstration
first_report = reports[0] if len(reports) > 0 else None


## Step 6: Export Report Data as JSON

Now let's export actual survey data from our example report. We'll start with the JSON format, which is great for programmatic analysis.

In [ ]:
# Export report data as JSON for analysis
firstReportId = reports[0]["Id"]
report_name = reports[0]["Name_EN"]

print(f"Exporting data from report: '{report_name}'")
print(f"Report ID: {firstReportId}")
print("API Call: POST /FieldSurveys/LdsStatistics/ExportDataAsJson")

# Request data export
reqResult = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/ExportDataAsJson',
    data={
        "authorities": [],      # Empty = all authorities (optional)
        "reportId": firstReportId,  # Mandatory: which report to export
        "surveyIds": [],       # Empty = all surveys (optional)  
        "speciesIds": []       # Empty = all species (optional)
    },
    cookies=session_cookies
)

# Parse the JSON response
response_data = reqResult.json()

print(f"Data export successful!")

# Convert the data to a pandas DataFrame for analysis
if 'data' in response_data:
    df_result = pd.DataFrame(response_data['data'])
    print(f"Retrieved {len(df_result)} data records")
    print(f"Data contains {len(df_result.columns)} columns:")
    print(list(df_result.columns))
    
    print(f"\n First 5 records from '{report_name}':")
    print("=" * 100)
    print(df_result.head())
    print("=" * 100)
else:
    print("⚠️ No 'data' field found in response")
    # print("📄 Response keys:", list(response_data.keys()))

## Step 7: Export Report Data as CSV

CSV format is perfect for sharing with colleagues or importing into Excel and other tools. Let's export the same data as a CSV file.

In [ ]:
# Export the same report data as CSV
print(f"Exporting '{report_name}' data as CSV file...")
print("API Call: POST /FieldSurveys/LdsStatistics/ExportDataAsCsv")

reqResult = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/ExportDataAsCsv',
    data={
        "authorities": [],          # Empty = all authorities (optional)
        "reportId": firstReportId,  # Mandatory: which report to export
        "surveyIds": [],           # Empty = all surveys (optional)
        "speciesIds": []           # Empty = all species (optional)
    },
    cookies=session_cookies
)

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Save the CSV data to file
output_filename = 'output/catchdata.csv'

with open(output_filename, 'w', encoding='utf-8') as file:
    file.write(reqResult.text)

print("CSV export successful!")
print(f"Data saved to: {output_filename}")

# Show some statistics about the exported file
import os
file_size = os.path.getsize(output_filename)

# Count lines in CSV (approximate record count)
with open(output_filename, 'r', encoding='utf-8') as file:
    line_count = sum(1 for line in file)
    
print(f"📈 Approximate records: {line_count-1:,} (excluding header)")
print(f"📁 File location: {os.path.abspath(output_filename)}")

## Step 8: Advanced Filtering - Export Specific Data

The API allows you to filter data by authorities, surveys, and species. This is useful when you need specific subsets of data rather than everything.

In [ ]:
# Example of filtered data export - specific authority and species
print(" Demonstrating filtered data export...")

# Use a different report for variety (if available)
if len(reports) > 1:
    second_report = reports[1]
    selected_report_id = second_report["Id"]
    selected_report_name = second_report["Name_EN"]
else:
    # Use the first report if only one is available
    selected_report_id = firstReportId
    selected_report_name = report_name

print(f" Using report: '{selected_report_name}'")
print(f" Filtering by authority: '{first_authority}'")

# Get a specific species ID for filtering (use first available species)
if len(species_df) > 0:
    # Try to find a species ID from our species data
    sample_species = species_df.iloc[0] if len(species_df) > 0 else None
    species_filter = [species_df.iloc[0].get('Id', 'd791be4d-593f-42f0-9a97-8c8d2181b9fe')] if sample_species is not None else ["d791be4d-593f-42f0-9a97-8c8d2181b9fe"]
    species_name = species_df.iloc[0].get('Species', 'Unknown') if sample_species is not None else 'Example Species'
    print(f"Filtering by species: '{species_name}'")
else:
    # Use example species ID
    species_filter = ["d791be4d-593f-42f0-9a97-8c8d2181b9fe"]
    print("Using example species filter")

print(" API Call: POST /FieldSurveys/LdsStatistics/ExportDataAsCsv (with filters)")

# Make filtered API request
reqResult = requests.post(
    baseUrl + 'FieldSurveys/LdsStatistics/ExportDataAsCsv',
    data={
        "authorities": [first_authority],    # Filter by specific authority
        "reportId": selected_report_id,      # Mandatory report ID  
        "surveyIds": [],                     # Could filter by specific surveys
        "speciesIds": species_filter         # Filter by specific species
    },
    cookies=session_cookies
)

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Save filtered results
filtered_filename = 'output/catchdata_specific_species.csv'

with open(filtered_filename, 'w', encoding='utf-8') as file:
    file.write(reqResult.text)
print("CSV export successful!")
print(f"Data saved to: {filtered_filename}")

Done!

You can now:

1. **Analyze the data** using pandas, Excel, or your preferred tools
2. **Modify filters** to get different subsets (change authorities, surveys, species)
3. **Integrate** this code into your own data processing workflows
